In [1]:
# install all packages to access .env and .local.env, and then access aws bedrock 

%pip install python-dotenv boto3 awscli Flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 895.4 kB/s eta 0:00:00:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.2/739.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
import json

load_dotenv(dotenv_path="../.env")

load_dotenv(dotenv_path="../.local.env")


import boto3

# get the aws credentials from the .local.env file in ../.local.env
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
accept = "application/json"
contentType = "application/json"


from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-east-1")


# Set the model ID, e.g., Mistral Large.
model_id = "mistral.mistral-large-2402-v1:0"

# Define the prompt for the model.
prompt = "Tell me a random story with a random topic"

# Embed the prompt in Mistral's instruction format.
formatted_prompt = f"<s>[INST] {prompt} [/INST]"

# Format the request payload using the model's native structure.
native_request = {
    "prompt": formatted_prompt,
    "max_tokens": 512,
    "temperature": 0.5,
}

# Convert the native request to JSON.
request = json.dumps(native_request)
def invoke_model(client, model_id, request, stream=False):
    try:
        if stream:
            # Invoke the model with the request and stream the response.
            streaming_response = client.invoke_model_with_response_stream(
                modelId=model_id, body=request
            )

            # Extract and print the response text in real-time.
            for event in streaming_response["body"]:
                chunk = json.loads(event["chunk"]["bytes"])
                if "outputs" in chunk:
                    print(chunk["outputs"][0].get("text"), end="")
        else:
            # Invoke the model with the request.
            response = client.invoke_model(modelId=model_id, body=request)

            # Decode the response body.
            model_response = json.loads(response["body"].read())

            # Extract and print the response text.
            response_text = model_response["outputs"][0]["text"]
            print(response_text)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

# Invoke the model and print the response.
invoke_model(client, model_id, request, True)

 Sure, I'd be happy to create a story for you. Let's explore a topic that's both random and interesting: "The Adventure of the Talking Tree in the City of Clocks."

Once upon a time, in a city known for its countless clocks, there was a unique tree that could talk. This wasn't an ordinary tree; it was a magnificent oak with branches that sprawled out like a wise old sage's arms, ready to share wisdom with anyone who would listen.

The city, called Horologium, was a bustling metropolis where time was the most valued commodity. The citizens were always rushing, their lives dictated by the endless ticking and tocking of the clocks that adorned every building, street corner, and even the smallest of alleyways.

Amidst this ceaseless clamor, the talking tree, whom the citizens named Tempus, stood in a quiet park. Tempus had a peculiar ability: it could slow down or speed up time within its immediate vicinity, creating a bubble of tranquility or a rush of hastened moments.

One day, a young 

 The purpose of a 'hello world' program is to provide a simple introduction to a programming language's syntax, demonstrating its basic input and output functions.